In [1]:
import math, time

# Possible Directions: up, down, left, right
MOVES = [(-1,0),(1,0),(0,-1),(0,1)]

def movement_grid(grid):
    """
    Scanning of the grid and identification of start, hospitals and obstacles.
    S - Start
    H - Hospitals
    C - Camp
    X - Restricted Areas
    """
    H_positions = []
    S = None
    C = None
    obstacles = set()
    free_cells = set()
    rows = len(grid)
    cols = len(grid[0]) if rows>0 else 0
    for i,row in enumerate(grid):
        for j,cell in enumerate(row):
            if cell == 'S':
                S = (i,j); free_cells.add((i,j))
            elif cell == 'C':
                C = (i,j); free_cells.add((i,j))
            elif cell == 'H':
                H_positions.append((i,j)); free_cells.add((i,j))
            elif cell == '0':
                free_cells.add((i,j))
            elif cell == 'X':
                obstacles.add((i,j))
            else:
                # If any other symbol (like empty), treat as free
                free_cells.add((i,j))
    return S, H_positions, C, obstacles, free_cells

def manhattan_distance(a,b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

def heuristic(pos, remaining_H, camp):
    """
    If no hospitals remain, estimate distance to camp.
    Else, estimate = distance from current pos to nearest hospital + distance from that hospital to camp.
    """
    if not remaining_H:
        return manhattan_distance(pos, camp)
    # Distance to nearest unvisited hospital
    d_to_h = min(manhattan_distance(pos, h) for h in remaining_H)
    # Distance from that hospital to camp (choosing nearest hospital to camp)
    d_h_to_c = min(manhattan_distance(h, camp) for h in remaining_H)
    return d_to_h + d_h_to_c

class Node:
    def __init__(self, pos, visited, g, f, parent=None):
        self.pos = pos
        self.visited = visited  # frozenset of visited hospital positions
        self.g = g              # cost so far
        self.f = f              # f = g + h estimate
        self.parent = parent

def depth(node):
    "Compute depth of node in search tree (for space complexity)."
    d = 0
    while node.parent:
        node = node.parent
        d += 1
    return d

def RBFS(node, f_limit, nodes_expanded, max_depth, H_positions, camp, free_cells):
    """
    Recursive Best-First Search (RBFS) for multi-goal A*.
    Returns (solution_node or None, new_f_value, nodes_expanded, max_depth).
    """
    # If goal condition met (at camp and all hospitals visited)
    if node.pos == camp and set(H_positions).issubset(node.visited):
        return node, node.f, nodes_expanded, max_depth

    # Generate successors
    successors = []
    r,c = node.pos
    for dr,dc in MOVES:
        nr, nc = r+dr, c+dc
        newpos = (nr,nc)
        if newpos not in free_cells:
            continue  # skip obstacles or out-of-bounds
        # Mark hospital visited if newpos is a hospital
        newvisited = set(node.visited)
        if newpos in H_positions:
            newvisited.add(newpos)
        newvisited = frozenset(newvisited)
        new_g = node.g + 1
        # Compute heuristic
        rem_H = [h for h in H_positions if h not in newvisited]
        new_h = heuristic(newpos, rem_H, camp)
        new_f = new_g + new_h
        succ = Node(newpos, newvisited, new_g, new_f, node)
        successors.append(succ)

    if not successors:
        return None, math.inf, nodes_expanded, max_depth  # dead end

    # Order by best f
    successors.sort(key=lambda x: x.f)
    # Recursive exploration
    while True:
        best = successors[0]
        if best.f > f_limit:
            return None, best.f, nodes_expanded, max_depth
        alt = successors[1].f if len(successors) > 1 else math.inf
        nodes_expanded += 1
        # Recurse on best with new limit (min of f_limit, second-best f)
        result, new_f, nodes_expanded, max_depth = RBFS(
            best, min(f_limit, alt), nodes_expanded, max(max_depth, depth(best)),
            H_positions, camp, free_cells
        )
        successors[0].f = new_f
        successors.sort(key=lambda x: x.f)
        if result:
            return result, new_f, nodes_expanded, max_depth

def main_medical_kit_planner():
    # Read input grid from user
    dims = input("Enter grid dimensions (rows cols): ").split()
    rows, cols = int(dims[0]), int(dims[1])
    grid = []
    print("Enter each row (with characters S,H,C,X,0):")
    for _ in range(rows):
        row = input().strip()
        grid.append(row)

    S, H_positions, C, obstacles, free_cells = movement_grid(grid)
    if not S or not C:
        print("Invalid input: Start (S) or Camp (C) missing.")
        return

    # Initialize start node
    start_node = Node(
        S, 
        frozenset([S] if S in H_positions else []),
        g=0, 
        f=0
    )
    start_node.f = start_node.g + heuristic(S, H_positions, C)

    # Run RBFS with timing
    nodes_expanded = 1
    max_depth = 0
    t0 = time.time()
    result, f_val, nodes_expanded, max_depth = RBFS(
        start_node, math.inf, nodes_expanded, 0, H_positions, C, free_cells
    )
    t1 = time.time()

    if result:
        # Reconstruct path
        path = []
        node = result
        while node:
            path.append(node.pos)
            node = node.parent
        path = list(reversed(path))
        cost = result.g
        print("\nOptimal path (sequence of coordinates):")
        for step in path:
            print(step, end=" ")
        print(f"\nTotal cost: {cost}")
    else:
        print("No valid route exists.")

    # Complexity instrumentation
    print(f"Nodes expanded (time complexity): {nodes_expanded}")
    print(f"Max search depth (space complexity): {max_depth}")
    print(f"Time taken: {t1-t0:.6f} seconds")


In [2]:
main_medical_kit_planner()

Enter grid dimensions (rows cols):  5 5


Enter each row (with characters S,H,C,X,0):


 S0HX0
 0X000
 H0H0X
 0H0X0
 0X00C



Optimal path (sequence of coordinates):
(0, 0) (0, 1) (0, 2) (1, 2) (2, 2) (2, 1) (2, 0) (3, 0) (3, 1) (3, 2) (4, 2) (4, 3) (4, 4) 
Total cost: 12
Nodes expanded (time complexity): 241
Max search depth (space complexity): 12
Time taken: 0.008370 seconds
